In [1]:
import tensorflow as tf
import numpy as np
seed = 123
np.random.seed(seed)
import cifar10_utils
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
from keras.applications import VGG16
import itertools

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

/home/hasan/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hasan/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hasan/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hasan/anaconda3/envs/tf_gpu/lib/python3.7/s

In [6]:
import tensorflow as tf
import os
from keras.backend.tensorflow_backend import set_session
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9

set_session(tf.Session(config=config))

In [7]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )

num_classes = 10
image_shape = x_train.shape[1:]

#training parameters
batch_size = 100
base_epochs = 1
baseMapNum = 32
weight_decay = 1e-4
class_names = ['other', 'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


model = cifar10_utils.create_base_model(image_shape, baseMapNum, weight_decay, 'kaggle')
model.summary()
initial_weights = model.get_weights()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [8]:
print('===============BASELINE Results On All Classes===========')

num_classes_excluding_other = 10
full_class_ids = np.arange(1, 11)  # 1-10

full_target_class_ids = full_class_ids[:num_classes_excluding_other]
selected_class_names = [class_names[el] for el in full_target_class_ids]
print('selected classes..: ' + str(selected_class_names))

cur_target_class_ids = np.copy(full_target_class_ids)

n_running_instance = 5
baseline_results = []
for i in range(n_running_instance):
    print('Running Instance ' + str(i) + ': ')
    baseline_result,_,_ = cifar10_utils.one_run(model, x_train, y_train, x_test, y_test, batch_size,full_target_class_ids, initial_weights, base_epochs, selected_class_names)
    baseline_results += [baseline_result]
baseline_results = np.array(baseline_results)


for i, class_name in enumerate(selected_class_names):
    plt.plot(np.arange(n_running_instance), baseline_results[:, i], label=selected_class_names)
plt.title('accuracy in different running intances (same data and config)')
plt.xlabel('running instance')
plt.ylabel('accuracy')
plt.legend()
plt.show()

# variance for each class among running instances
baseline_results_var = np.var(baseline_results, axis=0)
# plot variances of each class on different instances
plt.hist2d(range(len(selected_class_names)), baseline_results_var)
plt.title('Variances among running instances')
plt.xlabel(str(selected_class_names))
plt.ylabel('Variance')
plt.show()

===============BASELINE Results On All Classes===========
selected classes..: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Running Instance 0: 

======Training From Scratch=========

num of class 10
Epoch 1/3
50000/50000 [==============================] - 13s 252us/step - loss: 1.4670 - acc: 0.5231
Epoch 2/3
50000/50000 [==============================] - 12s 230us/step - loss: 0.8523 - acc: 0.7196
Epoch 3/3
50000/50000 [==============================] - 12s 231us/step - loss: 0.6593 - acc: 0.7932
Epoch 1/1
50000/50000 [==============================] - 13s 253us/step - loss: 0.4126 - acc: 0.8778
Epoch 1/1
1000/1000 [==============================] - 0s 429us/step
y_sum 840
1000/1000 [==============================] - 0s 91us/step
y_sum 1677
1000/1000 [==============================] - 0s 90us/step
y_sum 2646
1000/1000 [==============================] - 0s 91us/step
y_sum 3365
1000/1000 [==============================] - 0s 95us/step
y_sum 4

IndexError: index 10 is out of bounds for axis 1 with size 10

In [ ]:
num_classes_excluding_other = 3

full_target_class_ids = full_class_ids[:num_classes_excluding_other]
selected_class_names = [class_names[el] for el in full_target_class_ids]
print('selected classes: ' + str(selected_class_names))

full_target_class_ids_all_permutation = list(itertools.permutations(full_target_class_ids))

cur_target_class_ids = np.copy(full_target_class_ids)
# (cur_train_data, cur_train_labels) = cifar10_utils.generate_data(cur_target_class_ids, full_target_class_ids, 
#                                                                x_train, y_train, False, seed)

#TODO: temp testing code
# all_data = list(zip(cur_train_data, cur_train_labels))
# np.random.shuffle(all_data)
# cur_train_data = np.array([i[0] for i in all_data])
# cur_train_labels = np.array([i[1] for i in all_data])

GC_refining_results = []
GC_results = []
from_scratch_results = []
for full_target_class_ids_eli in full_target_class_ids_all_permutation: 
    print('sequence of class addition: ' + str([class_names[el] for el in full_target_class_ids_eli]))
    from_scratch_result, GC_refining_result, GC_result = cifar10_utils.one_run(model, x_train, y_train, 
                                                x_test, y_test, batch_size, full_target_class_ids_eli, 
                                                initial_weights, base_epochs, class_names, GC=True)
    from_scratch_results += [from_scratch_result]
    GC_refining_results += [GC_refining_result]
    GC_results += [GC_result]

In [5]:
print(from_scratch_results)

[0.7769999980926514, 0.864000004529953, 0.8589999973773956]
